In [1]:
%load_ext autoreload
%autoreload 2

import time

from utils import *
from mont import *

PAIR = 1  # physical core ID
CORES = scan_ht_cores()[PAIR]
platform = get_uarch()
MONT_BASE = f'{BUILD_DIR}/mont-'
KASLR_BIN = f'{BUILD_DIR}/kaslr'
OSSL_DIR = 'ossl'
RES_DIR = Path('results')
VER = 'bino'

width = 80
period = 200
n_samples = 70
drop_rate = .9
anom_thresh = 1000

if not RES_DIR.exists():
    RES_DIR.mkdir()
assert(RES_DIR.is_dir())

def run_mont(version='bino', smt_pair=0, ossl_dir=OSSL_DIR, data_file='-', oracle_file='ts.tsv'):
    cores = scan_ht_cores()[smt_pair]
    bin_path = MONT_BASE + version
    cmd = [bin_path, 'mont', cores[0], cores[1], ossl_dir]
    return run_once(cmd, data_file, oracle_file)

In [3]:
# *** collect execution traces *** (skip it if you have collected)
# we will collect more traces than what we actually use 
# because some traces are not suitable for training
for niter, use in [(150, 'boundary'), (40, 'signals'), (150, 'eval')]:
    data_l = []  # measured raw latencies
    oracle_l = []  # oracle iteration boundaries
    print(use, niter)
    for _ in range(niter):
        data, oracle = run_mont(version=VER, smt_pair=PAIR)
        data_l.append(data)
        oracle_l.append(oracle)
        time.sleep(.1)
        save_raw_data(VER, use, data_l, oracle_l)

boundary 150
signals 40
eval 150


In [4]:
# *** train iteration boundary classifier *** (skip it if it's trained)
iters = 100
raw_results, oracles = load_raw_data(VER, 'boundary')
gen_boundary_training(iters, VER, width, period, drop_rate,
                      raw_results=raw_results, oracles=oracles,
                      anomaly_thresh=anom_thresh)
train_boundary_model(best_boundary_params, VER, width, period)

54Data trace contains too extreme anomalies
70Data trace contains too extreme anomalies
101
(472000, 160) (472000,)
-1:  75.81%
1:  24.19%
0.9572457627118645


RandomForestClassifier(max_depth=30, max_features='log2', min_samples_split=5,
                       n_estimators=400, n_jobs=-1)

In [5]:
# *** train signal classifier *** (skip it if it's trained)
iters = 30
raw_results, oracles = load_raw_data(VER, 'signals')
data, oracle = gen_signal_training(VER, n_samples=n_samples, period=period,
                                   iters=iters, raw_results=raw_results,
                                   oracles=oracles, anomaly_thresh=anom_thresh)
train_signal_model(best_signal_params, VER, n_samples=n_samples, period=period)

26Data trace contains too extreme anomalies
29(17100, 70) (17100,)
0.0:  49.74%
1.0:  50.26%
0.9900584795321637


RandomForestClassifier(max_depth=30, max_features='log2', min_samples_split=5,
                       n_estimators=400, n_jobs=-1)

In [6]:
# evaluate accuracy with 100 traces
corret_anom = True
iters = 100
results = []
ideal_results = []
wrongs = []
ideal_wrongs = []

data_l, oracle_l = load_raw_data(VER, 'eval')
b_model = load_boundary_model(VER, width, period)
s_model = load_signal_model(VER, n_samples=n_samples, period=period)
cnt, valid = 0, 0
while valid < iters and cnt < min(len(data_l), len(oracle_l)):
    data, oracle = data_l[cnt], oracle_l[cnt]
    cnt += 1

    try:
        dt = DataTrace(data, oracle, std_filter_fact(), margin=2*width, period=period,
                        correct_anomaly=corret_anom, anomaly_thresh=anom_thresh)
    except ValueError as e:
        print(e)
        continue

    b_pred = BoundaryPredictor(dt, b_model, width=width)
    try:
        b_pred.predict()
    except ValueError as e:
        print(e)
        acc = None
    else:
        s_pred = SignalPredictor(dt, s_model, b_pred.boundaries, n_samples=n_samples)
        s_pred.predict()
        acc = s_pred.acc
        results.append(s_pred.acc)
        wrongs.append((s_pred.wrong_stats, s_pred.off_by_one))

    if acc is not None:
        s_pred_ideal = SignalPredictor(dt, s_model, dt.iter_boundaries, n_samples=n_samples)
        s_pred_ideal.predict()
        ideal_results.append(s_pred_ideal.acc)
        ideal_wrongs.append((s_pred_ideal.wrong_stats, s_pred_ideal.off_by_one))
        print(f'{cnt}: Acc: {acc:.2%} Oracle Acc: {s_pred_ideal.acc:.2%}')
        valid += 1

with open('results/mont_acc.pickle', 'wb') as f:
    pickle.dump(dict(results=results, ideal_results=ideal_results,
                     wrongs=wrongs, ideal_wrongs=ideal_wrongs), f)

Predicted tick rate: 12869.43231441048; oracle: 12877.2357403545
1: 98.95% 99.12% (Oracle)
Predicted tick rate: 12999.780701754386; oracle: 13012.424792841442
2: 99.12% 99.12% (Oracle)
Predicted tick rate: 12876.344086021505; oracle: 12880.25
3: 99.47% 99.47% (Oracle)
Predicted tick rate: 12893.205944798301; oracle: 12890.014234875445
4: 99.65% 99.65% (Oracle)
Predicted tick rate: 12992.583732057416; oracle: 13020.769911504425
5: 97.72% 99.12% (Oracle)
Predicted tick rate: 12877.114427860697; oracle: 12883.588961010879
6: 98.60% 98.60% (Oracle)
Predicted tick rate: 12869.37354988399; oracle: 12862.128975265017
7: 99.30% 99.82% (Oracle)
Predicted tick rate: 13018.181818181818; oracle: 12822.95406360424
Boundaries are highly unreliable. 1
Predicted tick rate: 12863.44537815126; oracle: 12861.143872113676
9: 99.12% 99.12% (Oracle)
Predicted tick rate: 12859.825327510916; oracle: 12853.800711743772
10: 98.77% 98.95% (Oracle)
Predicted tick rate: 12885.677083333334; oracle: 12892.0897682911

In [8]:
# print accuracy summary

with open('results/mont_acc.pickle', 'rb') as f:
    _res = pickle.load(f)
    results = _res['results']
    ideal_results = _res['ideal_results']
    wrongs = _res['wrongs']
    ideal_wrongs = _res['ideal_wrongs']

f = [r for r, w in zip(results, wrongs) if not w[1]]
se = np.std(f) / np.sqrt(len(f) - 1)
mean = np.mean(f)
print(f'Accuracy: {mean:.2%} +- {2 * se:.2%} (P=0.95)')

se = np.std(ideal_results) / np.sqrt(len(ideal_results) - 1)
mean = np.mean(ideal_results)
print(f'Accuracy (Oracle): {mean:.2%} +- {2 * se:.2%} (P=0.95)')

Accuracy: 98.49% +- 0.22% (P=0.95)
Accuracy (Oracle): 98.98% +- 0.15% (P=0.95)
